In [1]:
from sentence_transformers import SentenceTransformer
import os
import numpy as np
from sklearn.preprocessing import Normalizer
import json
import tqdm
import pandas as pd

/home/joseacentic/centic/LLM/envPRUEBA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [11]:
#Guardar modelo en local
path  = os.path.join(os.getcwd(), "modelo")

model.save(path)

In [2]:
##Cargar el modelo: 
model = SentenceTransformer("./modelo")

In [11]:
sentences = ['hola']
embedding = model.encode(sentences)

print(np.linalg.norm(embedding, ord=2))

#No vienen  normalizados

normalizer = Normalizer(norm='l2')
normalized_embeddings = normalizer.transform(embedding)
print(normalized_embeddings)

#print(np.linalg.norm(normalized_embeddings[1], ord=2))

[[ 1.06094110e+00  2.18850970e-01  6.44328177e-01  3.25962842e-01
   9.23534632e-02 -3.82572800e-01 -6.98094308e-01 -1.11933827e+00
  -1.25079834e+00 -2.50428855e-01  6.39026582e-01 -1.19380258e-01
   8.32551777e-01  1.00622642e+00  4.40649152e-01 -7.22044647e-01
   1.99867010e+00 -9.44283187e-01 -1.00704515e+00  8.22988331e-01
  -1.43235520e-01 -7.10854590e-01  9.65484157e-02  4.34059590e-01
  -8.83795083e-01  2.82747000e-01  9.65541363e-01  6.42392576e-01
   3.35802466e-01  1.64012477e-01 -2.76146561e-01  6.44394100e-01
  -5.93403995e-01  1.29968619e+00 -4.36596602e-01 -3.63067776e-01
   4.76554066e-01 -5.84675789e-01 -3.02468300e-01  3.72629553e-01
   5.33223093e-01  4.01632816e-01  1.09253228e+00 -1.03045833e+00
  -1.87431681e+00  1.50203037e+00 -1.23890162e-01  3.48138213e-01
   5.90081632e-01  5.14593542e-01 -1.81818202e-01  6.67019784e-01
   8.31277847e-01 -6.09675944e-01 -4.68126059e-01 -1.91233411e-01
  -1.12975858e-01 -1.29023001e-01  6.82089090e-01  1.42535913e+00
  -4.64385

In [167]:
normalized_embeddings[0] @ normalized_embeddings[1]

0.99999994

In [3]:
#pruebas con los datos de llmcontext: 
#RUTA: './DATA/tickets.json'

with open("./DATA/tickets.json", 'r') as ticket_file:
    tickets = json.load(ticket_file)


In [4]:
#tickets[0]['comments'][0]['body']

#elimina tickets sin respuesta: 
t = []
for ticket in tickets: 
    if len(ticket['comments']) == 0:
        print(f'El ticket {ticket["id"]} no tiene respuesta')
    else: 
        t.append(ticket)


El ticket 24433 no tiene respuesta
El ticket 26356 no tiene respuesta
El ticket 23406 no tiene respuesta
El ticket 24054 no tiene respuesta
El ticket 24134 no tiene respuesta
El ticket 24431 no tiene respuesta
El ticket 24795 no tiene respuesta
El ticket 24850 no tiene respuesta
El ticket 24916 no tiene respuesta
El ticket 25118 no tiene respuesta
El ticket 25629 no tiene respuesta
El ticket 26110 no tiene respuesta
El ticket 26321 no tiene respuesta
El ticket 26357 no tiene respuesta


In [6]:
normalizer = Normalizer(norm='l2')

for i in t:
    i['emb_pregunta'] = normalizer.transform(model.encode([i['description']])).tolist()
    i['emb_respuesta'] = normalizer.transform(model.encode([i['comments'][0]['body']])).tolist()


In [135]:
#Guardamos los tickets con los embeddings: 
with open('./DATA/tickets_embeddings.json', 'w') as f:
    f.write(json.dumps(t, indent=4))

In [180]:
#hacer lo mismo con los articles: 
with open("./DATA/articles.json", 'r') as f: 
    articles = json.load(f)


In [182]:
articles[0]

{'id': 8764508351644,
 'html_url': 'https://support.connectif.ai/hc/es/articles/8764508351644-Workflows-Hot-Sale-2023',
 'created_at': '2023-05-11T08:11:52Z',
 'updated_at': '2023-05-11T11:47:18Z',
 'title': 'Workflows Hot Sale 2023',
 'locale': 'es',
 'body': 'Ya están disponibles nuevas plantillas Hot Sale de workflows para activar a golpe de clic en Connectif.\n\n \n\n### **Hot Sale - Suscripción doble: newsletter y push**\n\nGana oportunidades de captar suscriptores a tu newsletter y notificaciones push durante Hot Sale. \n\n![Suscripci_n_doble_newsletter_y_push-min.png](https://support.connectif.ai/hc/article_attachments/8764227271452)\n\nExtra tip: Para conocer todos los detalles sobre el workflow haga clic [aquí](https://support.connectif.ai/hc/es/articles/4404180719634-Suscripci%C3%B3n-doble-newsletter-push)\n\n \n\n### **Hot Sale - Cuenta atrás**\n\nGenera urgencia recordando el fin de Hot Sale con una floating bar en la que se muestre una cuenta atrás.\n\n![Cuenta_atr_s-min.p

In [184]:
articles_emb = []
for article in articles:
    article['emb_body'] = normalizer.transform(model.encode([article['body']])).tolist()
    articles_emb.append(article)


In [198]:
with open('./DATA/articles_embeddings.json', 'w') as f:
    f.write(json.dumps(articles_emb, indent=4))

In [195]:
#lectura del json de embeddings: 
with open('./DATA/tickets_embeddings.json', 'r') as f:
    tickets_emb = json.load(f)

In [199]:
with open('./DATA/articles_embeddings.json', 'r') as f:
    articles_emb = json.load(f)


In [169]:
np.array(tickets_emb[0]['emb_pregunta'][0]) @ np.array(tickets_emb[10]['emb_respuesta'][0])

0.689458898163384

In [281]:
print('PREGUNTA')
print(tickets_emb[7]['description'])
print('-------------------------------------')
print('RESPUESTA')
print(tickets_emb[7]['comments'][0]['body'])

#IDEA: Coger una pregunta de un ticket o inventarla en txt. generar un dataframe con todos los articulos: id | titulo | distancia ... ordenarlo de mayor a menor distancia y mostrar un head. 

def calculaDistancia(pregunta, limit):
    df = pd.DataFrame(columns = ['id','title', 'emb'])
    print(pregunta)
    for a in articles_emb:
        dic = {}
        dic['id'] = a['id']
        dic['title'] = a['title']
        dic['emb'] = a['emb_body']
        #df = df.append(dic, ignore_index = True)
        df = pd.concat([df, pd.DataFrame(dic, index=[0])], ignore_index=True)

    #convertimos la pregunta a embedding:
    emb_preg = normalizer.transform(model.encode([pregunta]))
    df['distancia'] = df['emb'].apply(lambda x: np.array(x) @ emb_preg[0])
    df = df.sort_values('distancia', ascending = False)
    return df.head(limit)
    


PREGUNTA
Buenos dias, 
Me gustaria saber el funcionamiento del disparador de email externo, ya que he encontradoa donde hace referencia dentro de su centro de ayuda.

Gracias de antemano, 

Un saludo
-------------------------------------
RESPUESTA
Buenos días, Paula.  
   
En el siguiente enlace, en la sección "2. Email", se explica de manera detallada el funcionamiento de los nodos disparadores tipo email:  
* [Nodos de tipo Disparador](https://support.connectif.ai/hc/es/articles/360015811079-Nodos-de-tipo-Disparador)

Si te surge alguna otra cuestión, no dudes en escribir.  
   
¡Saludos!  
**Camilo | Connectif Team**  



In [282]:
#normalizer.transform(model.encode(['Hola']))[0] @ normalizer.transform(model.encode(['Pino']))[0]
calculaDistancia('Cómo fidelizar a tus compradores de Hot Sale', 10)

Cómo fidelizar a tus compradores de Hot Sale


,id,title,emb,distancia
22,6049188771484,Acceso preferente a clientes de alto valor,"[0.0015633644070476294, -0.01800038479268551, ...",0.454417
7,7116448367260,Lead Nurturing Postventa,"[-0.014296120963990688, -0.007922088727355003,...",0.434838
8,7087098296604,Lead Nurturing Preventa,"[-0.018018247559666634, -0.010873244144022465,...",0.433456
244,7873107334812,[LBD] Crea una campaña de recomendaciones pers...,"[-0.018190104514360428, -0.01634187437593937, ...",0.422656
5,7752237540508,[LBD] Curso: arranca tus primeras estrategias ...,"[-0.037452831864356995, 0.004717279691249132, ...",0.419567
286,4407779452562,Campaña mensual de productos más vendidos,"[-0.032478056848049164, -0.010385653004050255,...",0.411821
245,6675706995602,Captación newsletter y cupón al añadir product...,"[0.005627719685435295, 0.005278471857309341, -...",0.407025
247,5952263342226,Campaña de email optimizada,"[-0.039966437965631485, -0.009596409276127815,...",0.406624
301,360016806199,Buenas prácticas en captación de suscripciones...,"[-0.09440644085407257, 0.031678058207035065, -...",0.403562
281,4415019275026,Alerta de producto rebajado 1/2,"[-0.020015861839056015, -0.012303473427891731,...",0.402792


In [294]:
calculaDistancia(tickets_emb[1]['description'], 10)
#print(tickets_emb[1]['comments'][0]['body'])

Buenas, esta opción de la foto hace que solo se dispare una vez por contacto.

Es para segmentar por idioma de la página, cuando entre una vez el contacto no quiero que vuelva a segmentarlo.

------------------
Enviado desde: https://eu6.connectif.cloud/#/dashboard


,id,title,emb,distancia
49,360016416999,Detectar y reaccionar a enlaces web creados fu...,"[-0.01939813233911991, -0.0056239827536046505,...",0.758837
296,360014785419,Atributo de seguimiento de clics a enlaces,"[-0.0319833904504776, -0.0020671053789556026, ...",0.748038
11,6707069914652,Suscritos a push,"[-0.010649258270859718, -0.0017103675054386258...",0.740009
10,6707114915484,Suscritos a SMS,"[-0.02863769605755806, 0.003165251575410366, -...",0.736217
143,360014659820,Contactos anónimos o identificados suscritos a...,"[-0.0528210811316967, 0.021069681271910667, -0...",0.735503
9,6707287825436,No se pueden perder,"[-0.012422827072441578, 0.0073416568338871, -0...",0.732471
204,360014652000,"Etiqueta de ""Contacto ha hecho login""","[-0.04609205201268196, 0.02325289323925972, -0...",0.731343
57,360015342560,Fichas de contacto y huella digital,"[-0.022733163088560104, -0.0010015079751610756...",0.724781
168,360014652060,"Etiqueta de ""Contacto se ha registrado""","[-0.04801347851753235, 0.030688196420669556, -...",0.724286
56,360015491240,Campos del contacto,"[-0.025523923337459564, -0.0014205698389559984...",0.721414
